In [8]:
# !pip install accelerate
# !pip install transformers==4.38.2 --upgrade --force-reinstall
# !pip uninstall -y numpy transformers tokenizers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import accelerate
print(accelerate.__version__)

import transformers
print(transformers.__version__)


/data/healthy-ml/scratch/yuexing/hf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.6.0
4.51.3


In [2]:
import pandas as pd

# Load the model predictions
output_path = "Centaur_Lab_Second_Round.csv"
df = pd.read_csv(output_path)
df

,Origin,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,ID_corr,sentence_number_corr,answer_corr,data_source_corr,REMOVED_Sentences,sentence_number_df3,step1_excerpts,question_options,Filtered_Sentences,New_Sentences
0,ID0002,1,0.833333333,REMOVED,1,REMOVED,0.833333333,REMOVED,REMOVED,REMOVED,...,ID0002,11,D,jama,6,11,1. A woman in her 60s with a history of hyperl...,What Would You Do Next?\n\nA: Perform left upp...,"['3. On examination, corrected visual acuity w...",['1. A woman in her 60s with a history of hype...
1,ID0003,1,1,REMOVED,1,1,1,REMOVED,NaN,NaN,...,ID0003,7,G,medxpert,2,7,1. A 20-year-old woman comes to the primary ca...,Which of the following components is essential...,"['3. Her medical history is unremarkable, and ...",['1. A 20-year-old woman comes to the primary ...
2,ID0007,REMOVED,1,1,1,1,REMOVED,1,NaN,NaN,...,ID0007,7,B,medbullets,2,7,1. A 72-year-old man presents to his primary c...,Which of the following diagnostic tests would ...,['1. A 72-year-old man presents to his primary...,['2. He has felt very weak every morning with ...
3,ID0009,REMOVED,1,1,1,1,1,REMOVED,REMOVED,NaN,...,ID0009,8,D,jama,3,8,1. A woman in her 30s presented with multiple ...,What Is Your Diagnosis?\n\nA: Blue rubber bleb...,['1. A woman in her 30s presented with multipl...,['2. The lesions had been present since childh...
4,ID0010,1,REMOVED,REMOVED,REMOVED,REMOVED,REMOVED,1,NaN,NaN,...,ID0010,7,H,medxpert,5,7,1. A 17-year-old high school student accidenta...,What is the proper method for transporting the...,['2. His teacher applied dressings and pressur...,['1. A 17-year-old high school student acciden...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,ID1995,1,0.75,REMOVED,REMOVED,0.75,NaN,NaN,NaN,NaN,...,ID1995,5,D,mmlu,2,5,1. A 17-year-old girl is brought to the emerge...,Which of the following types of drugs is the m...,"['3. Her temperature is 37.1°C (98.8°F), pulse...",['1. A 17-year-old girl is brought to the emer...
1296,ID1996,1,REMOVED,1,NaN,NaN,NaN,NaN,NaN,NaN,...,ID1996,3,D,mmlu,1,3,1. A 68-year-old female presents to the emerge...,The most likely etiologic organism is\n\nA. Cl...,['2. Today the patient is nauseated and less r...,['1. A 68-year-old female presents to the emer...
1297,ID1997,1,1,REMOVED,1,REMOVED,REMOVED,REMOVED,REMOVED,REMOVED,...,ID1997,12,B,medxpert,9,12,1. A 27-year-old woman presents with a 4-month...,What is the most appropriate next step in the ...,"['3. She denies nipple discharge.', '5. She is...",['1. A 27-year-old woman presents with a 4-mon...
1298,ID1998,0.5,0.5,0.5,0.5,0.5,0.5,REMOVED,REMOVED,NaN,...,ID1998,8,A,medbullets,2,8,1. A 6-month-old girl is brought to the pediat...,Which of the structures labeled in Figure A wo...,"['7. A physical examination is unremarkable.',...",['1. A 6-month-old girl is brought to the pedi...


In [3]:
model_id = "Qwen/Qwen1.5-14B-Chat"

import tiktoken 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [04:07<00:00, 30.94s/it]


In [4]:
import re
import torch

# 1) Prepare the single example
row = df.iloc[1]
qa_id = "Merge_Q1"
context_text = row["New_Sentences"]
question = row["question_options"].strip()

# 2) Build the user prompt
prompt = (
    "You are a clinical reasoning assistant. You will receive a patient case summary "
    "and a multiple-choice question.\n\n"
    f"Patient Context:\n{context_text}\n\n"
    f"Question and Options:\n{question}\n\n"
    "Please select the single most appropriate answer. Respond only in the following format:\n\n"
    "Answer: <LETTER>"
)

# 3) Wrap in messages and apply the chat template
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user",   "content": prompt}
]
# tokenize=False so we get the raw string; add_generation_prompt=True inserts the model’s stop token
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# 4) Tokenize and move to device
inputs = tokenizer(text, return_tensors="pt").to(model.device)

# 5) Generate
with torch.no_grad():
    output_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=10,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False
    )

# 6) Remove the prompt tokens so only your answer remains
gen_ids = output_ids[0][ inputs.input_ids.shape[1] : ]

# 7) Decode
output_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
print("Raw model output:\n", output_text)

# 8) Extract the letter
match = re.search(r"([A-J])\b", output_text)
letter = match.group(1) if match else None
print("Parsed answer letter:", letter)


/data/healthy-ml/scratch/yuexing/hf_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/healthy-ml/scratch/yuexing/hf_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/data/healthy-ml/scratch/yuexing/hf_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mas

Raw model output:
 Answer: E
Parsed answer letter: E


In [5]:
import re
import csv
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

# # 1. Load model & tokenizer
model_id = "Qwen/Qwen1.5-14B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)  # ensure CUDA

# 2. Prepare output CSV
output_path = "predictions.csv"
with open(output_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["qa_id", "predicted_answer"])
    csvfile.flush()

    # 3. Stream over entire DataFrame with tqdm
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating answers"):
        qa_id = f"Merge_Q{idx+1}"
        context_text = row["New_Sentences"]
        question = row["question_options"].strip()

        # 4. Build the prompt & wrap in chat template
        user_prompt = (
            "You are a clinical reasoning assistant. You will receive a patient case summary "
            "and a multiple-choice question.\n\n"
            f"Patient Context:\n{context_text}\n\n"
            f"Question and Options:\n{question}\n\n"
            "Please select the single most appropriate answer. Respond only in the following format:\n\n"
            "Answer: <LETTER>"
        )
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user",   "content": user_prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # 5. Tokenize (with attention mask)
        inputs = tokenizer(
            text,
            return_tensors="pt",
            return_attention_mask=True,
            padding=True,
            truncation=True
        ).to(model.device)

        # 6. Generate
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=10,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False
            )

        # 7. Strip off the prompt tokens
        gen_ids = output_ids[0][ inputs.input_ids.shape[1] : ]

        # 8. Decode & extract the letter
        output_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
        match = re.search(r"Answer:\s*([A-J])", output_text)
        letter = match.group(1) if match else ""

        # 9. Write to CSV & flush
        writer.writerow([qa_id, letter])
        csvfile.flush()


Generating answers: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1300/1300 [03:14<00:00,  6.69it/s]


In [7]:
import pandas as pd

# 1. Load your saved predictions
preds = pd.read_csv("predictions.csv")

# 2. Align with the original df (assumes same order or matching qa_id)
results = preds.copy()
results["answer_corr"] = df["answer_corr"].values
results["correct"] = results["predicted_answer"] == results["answer_corr"]

# 3. Ensure pandas will print every row
pd.set_option("display.max_rows", None)

# 4. Display the full results table
results


,qa_id,predicted_answer,answer_corr,correct
0,Merge_Q1,B,D,False
1,Merge_Q2,E,G,False
2,Merge_Q3,A,B,False
3,Merge_Q4,D,D,True
4,Merge_Q5,H,H,True
5,Merge_Q6,B,B,True
6,Merge_Q7,B,C,False
7,Merge_Q8,A,B,False
8,Merge_Q9,D,C,False
9,Merge_Q10,D,D,True


In [11]:
# Assuming you have already built `results` as:
# results["correct"] = results["predicted_answer"] == results["answer_corr"]

overall_acc = results["correct"].mean()
overall_std = results["correct"].std()

print(f"Overall accuracy       : {overall_acc:.2%}")
print(f"Overall std deviation  : {overall_std:.2%}")


Overall accuracy       : 45.69%
Overall std deviation  : 49.83%


In [10]:
# 2. Combine with ground truth and data source
results = preds.copy()
results["answer_corr"]      = df["answer_corr"].values
results["data_source_corr"] = df["data_source_corr"].values
results["correct"]          = results["predicted_answer"] == results["answer_corr"]

# 3. Group by data source and compute mean & std
stats = (
    results
    .groupby("data_source_corr")["correct"]
    .agg(mean="mean", std="std")
    .reset_index()
)

# 4. (Optional) format as percentages
stats["mean"] = stats["mean"].map("{:.2%}".format)
stats["std"]  = stats["std"].map("{:.2%}".format)

print(stats)

  data_source_corr    mean     std
0             jama  52.75%  49.97%
1       medbullets  44.44%  49.81%
2         medxpert  21.07%  40.84%
3             mmlu  66.32%  47.38%
